# STAGE 2 - Pre-Processing | Banking Dataset #Absolute

Kelompok 7: 

1. Fikrie Lazuardi 
2. Rian Dwi Haryono
3. Retno dwi
4. Wahyuni
5. Hidayat Yatul
6. Yunita Rachmawati
7. Retno Harindhi

In [ ]:
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from imblearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from imblearn import over_sampling
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 5
rcParams['lines.linewidth'] = 3
rcParams['xtick.labelsize'] = 'x-large'
rcParams['ytick.labelsize'] = 'x-large'
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import dataset train
import pandas as pd 
dftrain = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv', sep= ";")
pd.set_option('display.max_columns', None) #Menunjukan semua kolom
dftrain.sample(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
11680,51,admin.,married,secondary,no,37,no,yes,unknown,20,jun,206,2,-1,0,unknown,no
28214,31,management,single,tertiary,no,1120,yes,no,cellular,29,jan,217,2,239,1,other,no
22032,33,management,single,tertiary,no,12,no,no,cellular,20,aug,471,5,-1,0,unknown,no
41958,43,technician,single,unknown,no,11115,no,no,cellular,23,oct,476,1,74,2,success,yes
10640,39,admin.,divorced,secondary,no,952,yes,no,unknown,16,jun,62,1,-1,0,unknown,no


## Data Cleansing

### Handle Missing Values

In [ ]:
dftrain.isnull().any()  # dataset traincsv tidak ada missing value 

age          False
job          False
marital      False
education    False
default      False
balance      False
housing      False
loan         False
contact      False
day          False
month        False
duration     False
campaign     False
pdays        False
previous     False
poutcome     False
y            False
dtype: bool

#### Imputasi Value di Beberapa Kolom

In [ ]:
dftrain['job'].value_counts()

blue-collar      9732
management       9458
technician       7597
admin.           5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
unknown           288
Name: job, dtype: int64

In [ ]:
dftrain['education'].value_counts()

secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: education, dtype: int64

In [ ]:
dftrain[dftrain['balance'] < 0]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
25,44,admin.,married,secondary,no,-372,yes,no,unknown,5,may,172,1,-1,0,unknown,no
28,46,management,single,secondary,no,-246,yes,no,unknown,5,may,255,2,-1,0,unknown,no
36,25,blue-collar,married,secondary,no,-7,yes,no,unknown,5,may,365,1,-1,0,unknown,no
37,53,technician,married,secondary,no,-3,no,no,unknown,5,may,1666,1,-1,0,unknown,no
45,36,admin.,single,primary,no,-171,yes,no,unknown,5,may,242,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44626,31,services,single,secondary,no,-205,no,no,cellular,1,sep,481,1,579,1,failure,yes
44629,28,blue-collar,single,secondary,no,-46,yes,no,cellular,1,sep,199,1,92,14,success,yes
44836,33,blue-collar,married,primary,no,-195,no,no,unknown,20,sep,9,1,-1,0,unknown,no
44908,48,management,divorced,tertiary,no,-130,yes,no,cellular,29,sep,110,2,61,9,failure,no


In [ ]:
dftrain['contact'].value_counts()

cellular     29285
unknown      13020
telephone     2906
Name: contact, dtype: int64

In [ ]:
dftrain['job'].mask(dftrain['job'] == 'unknown', dftrain['job'].mode(), inplace = True)
dftrain['education'].mask(dftrain['education'] == 'unknown', dftrain['education'].mode(), inplace = True)
dftrain['balance'].mask(dftrain['balance'] < 0, dftrain['balance'].mean(), inplace = True)
dftrain['contact'].mask(dftrain['contact'] == 'unknown', dftrain['contact'].mode(), inplace = True)

Simpulan :

1) Berikut jumlah data unknown pada kolom.
- Kolom job dengan value unknown berjumlah 288, lalu digantikan dengan nilai modus.
- Value unknown pada kolom education sebanyak 1857, kemudian digantikan dengan nilai modus
- Kolom contact dengan value unknown sebanyak 13020, lalu digantikan dengan nilai modus.

2) Terdapat data minus pada kolom balance sebanyak 3766, hal ini membuat kita menggantikan nilai minus menjadi nilai median agar bisa dilanjutkan untuk di analisis.

3) Adapula data unknown pada kolom potcome yaitu sebanyak 36959 , karena data unknown terlalu besar maka kami memutusakan untuk tidak menghapus atau menggantikan nya. Saat ini kami akan tetap mempertahankanya sampai dengan feature importance.

### Handle Duplicated Data

In [ ]:
dftrain.duplicated().sum() 

0

In [ ]:
dftrain.duplicated().any()

False

Simpulan :

Tidak memiliki data yang duplikat

### Handle Outliers

In [ ]:
#mengelompokan data sesuai dengan tipe data 
numericals=['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categoricals=['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']

In [ ]:
from scipy import stats
print(f'Jumlah baris sebelum memfilter outlier: {len(dftrain)}')

filtered_entries = np.array([True] * len(dftrain))

for col in numericals:
    zscore = abs(stats.zscore(dftrain[col])) 
    filtered_entries = (zscore < 3) & filtered_entries 
    
dftrain = dftrain[filtered_entries] 

print(f'Jumlah baris setelah memfilter outlier: {len(dftrain)}')

Jumlah baris sebelum memfilter outlier: 45211
Jumlah baris setelah memfilter outlier: 40211


Simpulan :

Analisis ini menggunakan Z-score filtering (cenderung lebih konservatif)
karena cenderung lebih aman dan tidak terlalu berisiko. Dimana model cenderung lebih hati-hati untuk memprediksi apakah seseorang akan berlangganan atau tidak berlangganan

### Feature Transformation

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# kita tranformasi
dftrain['age_norm'] = MinMaxScaler().fit_transform(dftrain['age'].values.reshape(len(dftrain),1))
dftrain['balance_std'] = StandardScaler().fit_transform(dftrain['balance'].values.reshape(len(dftrain),1))
dftrain['days_norm'] = MinMaxScaler().fit_transform(dftrain['day'].values.reshape(len(dftrain),1))
dftrain['duration_norm'] = MinMaxScaler().fit_transform(dftrain['duration'].values.reshape(len(dftrain),1))
dftrain['campaign_std'] = StandardScaler().fit_transform(dftrain['campaign'].values.reshape(len(dftrain),1))
dftrain['pdays_norm'] = MinMaxScaler().fit_transform(dftrain['pdays'].values.reshape(len(dftrain),1))
dftrain['previous_norm'] = MinMaxScaler().fit_transform(dftrain['previous'].values.reshape(len(dftrain),1))

Simpulan :

Normalisasi scaller digunakan ketika datanya sudah cukup bagus dan tidak banyak outliersnya. Sedangkan, Standarisasi scaller digunakan saat datanya memiliki banyak outlier

### Feature Encoding

In [ ]:
def categorize(df):
    new_df = df.copy()
    le = preprocessing.LabelEncoder()
    
    new_df['job'] = le.fit_transform(new_df['job'])
    new_df['marital'] = le.fit_transform(new_df['marital'])
    new_df['education'] = le.fit_transform(new_df['education'])
    new_df['default'] = le.fit_transform(new_df['default'])
    new_df['housing'] = le.fit_transform(new_df['housing'])
    new_df['loan'] = le.fit_transform(new_df['loan'])
    new_df['month'] = le.fit_transform(new_df['month'])
    new_df['contact'] = le.fit_transform(new_df['contact'])
    new_df['poutcome'] = le.fit_transform(new_df['poutcome'])
    new_df['y'] = le.fit_transform(new_df['y'])
    return new_df

In [ ]:
dftrain = categorize(dftrain)
dftrain.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,age_norm,balance_std,days_norm,duration_norm,campaign_std,pdays_norm,previous_norm
0,58,4,1,2,0,2143.0,1,0,0,5,8,261,1,-1,0,3,0,0.740741,0.550385,0.133333,0.253398,-0.750442,0.0,0.0
1,44,9,2,1,0,29.0,1,0,2,5,8,151,1,-1,0,3,0,0.481481,-0.717479,0.133333,0.146602,-0.750442,0.0,0.0
2,33,2,1,1,0,2.0,1,1,2,5,8,76,1,-1,0,3,0,0.277778,-0.733672,0.133333,0.073786,-0.750442,0.0,0.0
3,47,1,1,3,0,1506.0,1,0,2,5,8,92,1,-1,0,3,0,0.537037,0.168347,0.133333,0.089320,-0.750442,0.0,0.0
4,33,11,2,3,0,1.0,0,0,2,5,8,198,1,-1,0,3,0,0.277778,-0.734272,0.133333,0.192233,-0.750442,0.0,0.0


Simpulan :
Feature encoding menggunakan jenis label encoder untuk memudahkan dalam pengolahan data dengan dilakukan perubahan value dari kategorikal menjadi numerik di beberapa kolom

### Feature Selection

In [ ]:
dftrain = dftrain.drop(columns = ['default','contact','poutcome','pdays_norm', 'previous_norm', 'days_norm','month'],axis=1)
dftrain = dftrain.drop(columns = ['age','balance', 'day','duration','campaign','pdays', 'previous'],axis =1 )

Simpulan :

Feature selection digunakan untuk menghilangkan fitur-fitur yang tidak relevan atau bahkan merugikan dalam pemodelan. Seperti, fitur-fitur yang tidak memiliki hubungan yang kuat dengan target atau fitur yang memiliki tingkat korelasi yang tinggi dengan fitur lain, yaitu pdays dan previous dengan days


### Handling Imbalance

In [ ]:
X = dftrain[[col for col in dftrain.columns if (str(dftrain[col].dtype) != 'object') and 
         col not in ['y']]]
y = dftrain['y'].values
print(X.shape)
print(y.shape)

(40211, 9)
(40211,)


In [ ]:
from imblearn import under_sampling, over_sampling
# X_under, y_under = under_sampling.RandomUnderSampler().fit_resample(X, y)
# X_over, y_over = over_sampling.RandomOverSampler().fit_resample(X, y)
X_over_SMOTE, y_over_SMOTE = over_sampling.SMOTE(random_state=42).fit_resample(X, y)

In [ ]:
print('Original')
print(pd.Series(y).value_counts())
# print('\n')
# print('OVERSAMPLING')
# print(pd.Series(y_over).value_counts())
print('\n')
print('SMOTE')
print(pd.Series(y_over_SMOTE).value_counts())

Original
0    36157
1     4054
dtype: int64


SMOTE
0    36157
1    36157
dtype: int64


Simpulan :

Adanya penggandaan data menggunakan cara oversampling dengan teknik SMOTE. Perubahan yang terjadi pada value 1 yaitu yes dari 4054 menjadi 36157



In [ ]:
dftrain.to_csv('stage2clear', index=False)